### MGSC 695 - Optimization for Data Science

#### Winter 2024, Dr.Sanjith Gopalakrishnan

### Assignment 2

#### Submitted by Jared Balakrishnan (McGill ID #261175926)

In [70]:
from pathlib import Path 
import gurobipy as gb
from gurobipy import *
import pandas as pd 
import numpy as np

In [71]:
dataset_path = Path().absolute() / "datasets"

def read_dataset(file_path: Path) -> pd.DataFrame:

    return pd.read_csv(file_path)

In [72]:
advertising_df = read_dataset(dataset_path / "advertising.csv")

In [73]:
advertising_df.head(5)

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


For a linear regression model, the L1 loss function can be written as:

$$
 y_i - (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) 
 $$

Choosing parameters such that the sum of absolute errors is minimized, ie minimizing the L1 loss function can therefore be written as:

$$
\text{minimize} \sum_{i=1}^{n} \left| y_i - (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) \right|
$$

The L1 loss function can be expressed as: 

$$
\left| y_i - (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) \right| = \text{max} \{ y_i - (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}), (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) - y_i \}
$$

Let's say $y_i - (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) = w$

We can therefore write:

$$
 w \geq \text{max} \{ y_i - (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}), (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) - y_i \}
 $$

 $$
 w \geq y_i - (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip})
 $$

  $$
 w \geq (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) - y_i
 $$

 Therefore we can formulate the linear program as being:
 
 $$
 \text{min}(w) \\
 \text{such that } w \geq y_i - (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) \\
 w \geq (\beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip}) - y_i
 $$

 Considering the dataset we are working with seems to be dealing with advertising, it's hard to imagine negative amounts of money being spent on any advertising medium, because of which we can also write a non-negativity constraint as follows:

 $$
 x_{ip} \geq 0 \\
 i = 1, 2, \cdots, n \\
 p = 1, 2, 3
 $$

## Setting up the Gurobi Model

In [74]:
linearRegModel = gb.Model("Linear Regression Model (L1 Loss Function)")

In [75]:
# Initializing Decision Variables

Beta = linearRegModel.addVars(4, vtype = GRB.CONTINUOUS, name = [f"Beta_{i}" for i in range(4)])
W = linearRegModel.addVar(lb = 0, vtype = GRB.CONTINUOUS)

In [76]:
# Initializing the expression for the objective function

linearRegModel.setObjective(W, GRB.MINIMIZE)

In [77]:
# Setting up the constraints

In [78]:
exp_pos: float  = 0

for index, row in advertising_df.iterrows():

    exp_pos += row['Sales'] - (Beta[0] + Beta[1]*row['TV'] + Beta[2]*row['Radio'] + Beta[3]*row['Newspaper'])

In [79]:
exp_neg: float  = 0

for index, row in advertising_df.iterrows():

    exp_neg += (Beta[0] + Beta[1]*row['TV'] + Beta[2]*row['Radio'] + Beta[3]*row['Newspaper']) - row['Sales']

In [80]:
linearRegModel.addConstr(W >= exp_pos, name = "Absolute Value - Positive")
linearRegModel.addConstr(W >= exp_neg, name = "Absolute Value - Negative")

<gurobi.Constr *Awaiting Model Update*>

In [81]:
linearRegModel.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 5 columns and 10 nonzeros
Model fingerprint: 0xda56eeea
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 3e+03]
Presolve removed 2 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [82]:
for v in linearRegModel.getVars():

    print(v.varName, v.x)

Beta_0 14.022500000000003
Beta_1 0.0
Beta_2 0.0
Beta_3 0.0
C4 0.0
